In [6]:
import torch
import torchvision

class TrainData:
    def __init__(self):
        self._download_train_data()
        
    def _download_train_data(self):
        orig_train = torchvision.datasets.CIFAR10('data', train=True, transform=torchvision.transforms.ToTensor(), download=True)

        classes = tuple(orig_train.classes)

        keep_labels = (classes.index('cat'), classes.index('dog'))  #tuple(range(len(classes)))
        aux_labels = tuple(classes.index(a) for a in ['airplane', 'automobile', 'bird', 'ship', 'truck'])
        n = 5000 * len(keep_labels)
        auxn = 25000
        channels = 3
        w = 32
        h = 32
        X = torch.zeros((n, channels, w, h))
        y = torch.zeros((n,), dtype=torch.long)
        auxX = torch.zeros((auxn, channels, w, h))

        j = 0
        auxj = 0
        for x,label in orig_train:
            if label in keep_labels:
                X[j,:,:,:] = x
                y[j] = keep_labels.index(label)
                j += 1
            if label in aux_labels:
                auxX[auxj,:,:,:] = x
                auxj += 1
        if j != n:
            raise Exception(f"Wrong number of valid examples {j} {n}")
        if auxj != auxn:
            raise Exception(f"Wrong number of aux examples {auxj} {auxn}")
        self._X = X
        self._y = y
        self._n = n
        self._auxX = auxX
        self._auxn = auxn
        self._n_classes = len(keep_labels)
        
    def in_distribution_dataset(self):
        return [(self._X[i], self._y[i]) for i in range(self._n)]
        
    def mixed_dataset(self):
        ind = [(self._X[i], (self._y[i], 0)) for i in range(self._n)]
        ood = [(self._auxX[i], (0.5, 1)) for i in range(self._auxn)]
        return ind + ood
    
    def n_classes(self):
        return self._n_classes
        
train_data = TrainData()
print("Have training data")

Files already downloaded and verified
Have training data


In [7]:

class TestData:
    def __init__(self):
        self._download_test_data()
        
    def _download_test_data(self):
        orig_train = torchvision.datasets.CIFAR10('data', train=False, transform=torchvision.transforms.ToTensor(), download=True)

        classes = tuple(orig_train.classes)

        keep_labels = (classes.index('cat'), classes.index('dog'))   # tuple(range(len(classes)))
        ood_labels = tuple(classes.index(a) for a in ['deer','frog','horse'])
        n = 1000 * len(keep_labels)
        oodn = 3000
        channels = 3
        w = 32
        h = 32
        X = torch.zeros((n, channels, w, h))
        y = torch.zeros((n,), dtype=torch.long)
        oodX = torch.zeros((oodn, channels, w, h))

        j = 0
        oodj = 0
        for x,label in orig_train:
            if label in keep_labels:
                X[j,:,:,:] = x
                y[j] = keep_labels.index(label)
                j += 1
            if label in ood_labels:
                oodX[oodj,:,:,:] = x
                oodj += 1
        if j != n:
            raise Exception(f"Wrong number of valid examples {j} {n}")
        if oodj != oodn:
            raise Exception(f"Wrong number of ood examples {oodj} {oodn}")
        self._X = X
        self._y = y
        self._n = n
        self._oodX = oodX
        self._oodn = oodn
        
    def in_distribution_dataset(self):
        return [(self._X[i], self._y[i]) for i in range(self._n)]
        
#    def mixed_dataset(self):
#        ind = [(self._X[i], (self._y[i], 0)) for i in range(self._n)]
#        ood = [(self._auxX[i], (0.5, 1)) for i in range(self._auxn)]
#        return ind + ood
        
test_data = TestData()
print("Have test data")

Files already downloaded and verified
Have test data


In [40]:
class SimpleCnnModel(torch.nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(32, 64, 3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(64, 64, 4),
            torch.nn.ReLU(),
            torch.nn.Flatten(),
            torch.nn.Linear(576, n_classes),
            #torch.nn.ReLU(),
            #torch.nn.Linear(64, n_classes),
        )

    def forward(self, x):
        return self.layers(x)


In [41]:
class Experiment:
    def __init__(self):
        global train_data
        global test_data
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = SimpleCnnModel(train_data.n_classes()).to(self.device)
        self.dataloader = torch.utils.data.DataLoader(
            train_data.in_distribution_dataset(),
            batch_size=64,
            shuffle=True
        )
        self.test_dataloader = torch.utils.data.DataLoader(
            test_data.in_distribution_dataset(),
            batch_size=64,
            shuffle=False
        )
        self.loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')
        self.optimizer = torch.optim.Adam(self.model.parameters(), weight_decay=0.4, lr=0.001)
        self.num_epochs = 100
        
    def _train(self):
        print(self.device)
        for epoch in range(self.num_epochs):
            #print(f"=== Epoch {epoch}===")
            running_loss = torch.zeros(())
            train_accuracy = torch.zeros(())
            running_count = 0
            for inputs, labels in self.dataloader:
                self.optimizer.zero_grad()
                outputs = self.model(inputs.to(self.device))
                predictions = outputs.detach().argmax(dim=1)
                loss = self.loss_fn(outputs, labels.to(self.device))
                loss.backward()
                self.optimizer.step()
                running_loss += loss.detach().to('cpu')
                train_accuracy += (predictions.to('cpu') == labels).sum()
                running_count += outputs.shape[0]
                    
            #print('    Loss', running_loss.item() / running_count)  

            if True:
                test_accuracy = torch.zeros(())
                test_count = 0
                with torch.no_grad():
                    for inputs, labels in self.test_dataloader:
                        outputs = self.model(inputs.to(self.device))
                        predictions = outputs.argmax(dim=1)
                        test_accuracy += (predictions.to('cpu') == labels).sum()
                        test_count += outputs.shape[0]
                print('Epoch', f'{epoch:2}',
                      'Loss', f'{running_loss.item() / running_count:20}',
                      'Train', f'{train_accuracy.item() / running_count:10}',
                      'Test', test_accuracy.item() / test_count)

In [ ]:
Experiment()._train()

cuda:0
Epoch  0 Loss    0.679515380859375 Train     0.5791 Test 0.6145
Epoch  1 Loss     0.65369501953125 Train      0.617 Test 0.635
Epoch  2 Loss         0.6317984375 Train     0.6417 Test 0.6625
Epoch  3 Loss        0.61188671875 Train     0.6629 Test 0.668
Epoch  4 Loss     0.59531572265625 Train     0.6805 Test 0.6705
Epoch  5 Loss     0.58049169921875 Train      0.694 Test 0.699
Epoch  6 Loss    0.569049560546875 Train     0.7016 Test 0.671
Epoch  7 Loss    0.564333642578125 Train     0.7061 Test 0.7145
Epoch  8 Loss    0.549903173828125 Train     0.7169 Test 0.7065
Epoch  9 Loss     0.54665927734375 Train     0.7209 Test 0.7145
Epoch 10 Loss     0.53787626953125 Train     0.7279 Test 0.712
Epoch 11 Loss     0.52879833984375 Train      0.734 Test 0.725
Epoch 12 Loss     0.52554931640625 Train     0.7396 Test 0.728
Epoch 13 Loss     0.52233583984375 Train     0.7378 Test 0.702
Epoch 14 Loss        0.51834140625 Train     0.7387 Test 0.6835
Epoch 15 Loss     0.51278232421875 Train 